In [6]:
# Connect to the database
import sqlalchemy
from sqlalchemy import create_engine

engine = create_engine('postgresql+psycopg2://ctang:@localhost:5432/ctang', encoding='utf-8') # no password
connection = engine.connect()

In [10]:
# Set up the tables
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Enum, Boolean, Integer, Numeric, Text

metadata = MetaData()

orders = Table('orders', metadata,
    Column('order_id', Integer(), autoincrement=True, primary_key=True),
    Column('user_id', Integer(), autoincrement=True),
    Column('order_eval_set', Enum('prior', 'train', 'test')),
    Column('order_number', Integer()),
    Column('order_dow', Integer()),
    Column('order_hour_of_day', Integer()),
    Column('days_since_prior', Numeric(), nullable=True)
)

aisles = Table('aisles', metadata,
    Column('aisle_id', Integer(), autoincrement=True, primary_key=True),
    Column('aisle', Text)
)

departments = Table('departments', metadata,
    Column('department_id', Integer(), autoincrement=True, primary_key=True),
    Column('department', Text)
)

products = Table('products', metadata,
    Column('product_id', Integer(), autoincrement=True, primary_key=True),
    Column('product_name', Text()),
    Column('aisle_id', ForeignKey('aisles.aisle_id')),
    Column('department_id', ForeignKey('departments.department_id'))
)

order_products__train = Table('order_products__train', metadata,
    Column('order_id', ForeignKey('orders.order_id')),
    Column('product_id', ForeignKey('products.product_id')),
    Column('add_to_cart_order', Integer(), autoincrement=True),
    Column('reordered', Boolean())
)

In [11]:
from sqlalchemy.sql import select, func, desc

# Top 10 departments with the most product listings
columns = [departments.c.department, func.count(products.c.product_id).label('product_count')]
s = select(columns) \
    .select_from(departments.join(products)) \
    .group_by(departments.c.department) \
    .order_by(desc('product_count'))

rp = connection.execute(s)
print "%20s | %s" % ('department', 'number of products')
print '-' * 50
for record in rp:
    print "%20s | %s" % (record.department, record.product_count)

          department | number of products
--------------------------------------------------
       personal care | 6563
              snacks | 6264
              pantry | 5371
           beverages | 4365
              frozen | 4007
          dairy eggs | 3449
           household | 3085
        canned goods | 2092
     dry goods pasta | 1858
             produce | 1684
              bakery | 1516
                deli | 1322
             missing | 1258
       international | 1139
           breakfast | 1115
              babies | 1081
             alcohol | 1054
                pets | 972
        meat seafood | 907
               other | 548
                bulk | 38


In [102]:
# # /* Top 10 most ordered items */
# columns = [order_products__train.c.product_id, func.count().label('num_ordered')]
# product_counts = select(columns) \
#     .select_from(order_products__train) \
#     .group_by(order_products__train.c.product_id) \
#     .order_by(desc('num_ordered')) \
#     .limit(10) \
#     .subquery('product_counts')

# col2 = [products.c.product_name, product_counts.c.num_ordered]
# r = select(col2).select_from(product_counts).join(products).limit(10)
# print str(product_counts)
# rp = connection.execute(s)
# for record in rp:
#     print record
    
# # SELECT p.product_name, o.num_ordered, a.aisle
# #   FROM (SELECT o.product_id as product_id, COUNT(*) AS num_ordered
# #           FROM order_products__train o
# #       GROUP BY o.product_id
# #       ORDER BY num_ordered DESC) AS o
# #   JOIN products p ON p.product_id = o.product_id
# #   JOIN aisles a ON p.aisle_id = a.aisle_id
# #  LIMIT 10;

In [16]:
# Connect to the database
import sqlalchemy
from sqlalchemy import create_engine

engine = create_engine('postgresql+psycopg2://ctang:@localhost:5432/ctang', encoding='utf-8') # no password
connection = engine.connect()

from sqlalchemy import Table, Column, Enum, Boolean, Integer, Numeric, Text, Unicode, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

Base = declarative_base()

class Aisle(Base):
    __tablename__ = 'aisles'
    
    aisle_id = Column('aisle_id', Integer(), autoincrement=True, primary_key=True)
    aisle = Column('aisle', Text)
    
    def __repr__(self):
        return "Aisle(aisle_id={self.aisle_id}, aisle={self.aisle})".format(self=self)

class Department(Base):
    __tablename__ = 'departments'
    
    department_id = Column('department_id', Integer(), autoincrement=True, primary_key=True)
    department = Column('department', Text)
    
    def __repr__(self):
        return "Department(department_id={self.department_id}, department={self.department})".format(self=self)
    
class Product(Base):
    __tablename__ = 'products'
    
    product_id = Column('product_id', Integer(), autoincrement=True, primary_key=True)
    product_name = Column('product_name', Text())
    aisle_id = Column(Integer(), ForeignKey('aisles.aisle_id'))
    department_id = Column(Integer(), ForeignKey('departments.department_id'))
    
    aisle = relationship('Aisle', backref=backref('products', order_by=product_id))
    department = relationship('Department', backref=backref('products', order_by=product_id))
    
    def __repr__(self):
        return u"Product(product_id={self.product_id}, " \
                "product_name={self.product_name}, " \
                "aisle_id={self.aisle_id}, " \
                "department_id={self.department_id})".format(self=self)

class Order(Base):
    __tablename__ = 'orders'
    
    order_id = Column('order_id', Integer(), autoincrement=True, primary_key=True)
    user_id = Column('user_id', Integer())
    order_eval_set = Column('order_eval_set', Enum('prior', 'train', 'test', name='order_eval_set'))
    order_number = Column('order_number', Integer())
    order_dow = Column('order_dow', Integer())
    order_hour_of_day = Column('order_hour_of_day', Integer())
    days_since_prior = Column('days_since_prior', Numeric(), nullable=True)

    def __repr__(self):
        return 'Order(order_id={self.order_id}, ' \
                'user_id={self.user_id}, ' \
                'order_eval_set={self.order_eval_set}, ' \
                'order_number={self.order_number}, ' \
                'order_dow={self.order_dow}, ' \
                'order_hour_of_day={self.order_hour_of_day}, ' \
                'days_since_prior={self.days_since_prior})'.format(self=self)

class LineItem(Base):
    __tablename__ = 'order_products__train'
    
    order_id = Column(Integer(), ForeignKey('orders.order_id'), primary_key=True)
    product_id = Column(Integer(), ForeignKey('products.product_id'), primary_key=True)
    add_to_cart_order = Column('add_to_cart_order', Integer())
    reordered = Column('reordered', Boolean())
    
    order = relationship('Order', backref=backref('line_items', order_by=add_to_cart_order))
    department = relationship('Product', backref=backref('line_items', order_by=add_to_cart_order))
    
    def __repr__(self):
        return 'LineItem(order_id={self.order_id}, ' \
                'product_id={self.product_id}, ' \
                'add_to_cart_order={self.add_to_cart_order}, ' \
                'reordered={self.reordered})'.format(self=self)

Base.metadata.create_all(engine)

In [19]:
# product 24 has a non-ASCII character in it. Print can't understand it.
# What's the alternative to print in ASCII?
product = session.query(Product).filter(Product.product_id == 24).first()

24


In [29]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

from sqlalchemy.sql import func, desc

# Top 10 departments with the most product listings
for record in session.query(Department.department, func.count().label('product_count')) \
    .join(Product) \
    .group_by(Department.department) \
    .order_by(desc('product_count')) \
    .limit(10):
        print "%20s | %s" % (record.department, record.product_count)  

       personal care | 6563
              snacks | 6264
              pantry | 5371
           beverages | 4365
              frozen | 4007
          dairy eggs | 3449
           household | 3085
        canned goods | 2092
     dry goods pasta | 1858
             produce | 1684


In [32]:
# Top 10 most ordered items
for record in session.query(Product.product_name, func.count().label('num_orders')) \
    .join(LineItem) \
    .group_by(Product.product_id) \
    .order_by(desc('num_orders')) \
    .limit(10):
        print "%30s | %s" % (record.product_name, record.num_orders)  

                        Banana | 18726
        Bag of Organic Bananas | 15480
          Organic Strawberries | 10894
          Organic Baby Spinach | 9784
                   Large Lemon | 8135
               Organic Avocado | 7409
          Organic Hass Avocado | 7293
                  Strawberries | 6494
                         Limes | 6033
           Organic Raspberries | 5546
